## Dataset

In [1]:
import time
import random as rd
import csv

In [2]:
def crea_dataset():
    column_names=['server_id', 'application_id', 'start_timestamp']
    server_list = [x for x in range(1,11)]
    application_list = ['app'+ str(x)  for x in range(1,11)]
    start_time_list=[int(time.time())+x for x in range(1,1000)]
    data=[]
    data.append(column_names)
    for i in range(1000):
        row=[i,rd.choice(server_list),rd.choice(application_list), rd.choice(start_time_list)]
        data.append(row)
    with open('dataset.csv', 'w') as f:
        writer = csv.writer(f)
        writer.writerows(data)
    return data    

In [3]:
crea_dataset();

## Analysis

In [4]:
import numpy as np
import pandas as pd

In [5]:
df= pd.DataFrame.from_csv('dataset.csv')

In [6]:
df.head()

,server_id,application_id,start_timestamp
0,10,app9,1508436418
1,6,app4,1508436121
2,4,app7,1508435571
3,10,app4,1508436073
4,9,app6,1508435725


In [7]:
df.columns

Index(['server_id', 'application_id', 'start_timestamp'], dtype='object')

In [8]:
df=df.sort_values(by=['start_timestamp'], ascending=[True])

In [9]:
df = df.reset_index(drop=True)
df[:20]

,server_id,application_id,start_timestamp
0,10,app10,1508435557
1,9,app8,1508435557
2,1,app5,1508435559
3,8,app6,1508435559
4,5,app4,1508435560
5,6,app10,1508435563
6,8,app4,1508435563
7,8,app10,1508435565
8,7,app4,1508435565
9,8,app10,1508435566


In [10]:
df.groupby(df['server_id']).describe()

start_timestamp                                          \
                    count          mean         std           min   
server_id                                                           
1                    89.0  1.508436e+09  288.866675  1.508436e+09   
2                   122.0  1.508436e+09  303.829318  1.508436e+09   
3                    98.0  1.508436e+09  293.421401  1.508436e+09   
4                   107.0  1.508436e+09  294.011363  1.508436e+09   
5                    93.0  1.508436e+09  285.021643  1.508436e+09   
6                   108.0  1.508436e+09  289.458248  1.508436e+09   
7                    86.0  1.508436e+09  298.439935  1.508436e+09   
8                   100.0  1.508436e+09  286.462731  1.508436e+09   
9                   105.0  1.508436e+09  291.049675  1.508436e+09   
10                   92.0  1.508436e+09  306.785896  1.508436e+09   

                                                                   
                    25%           50%           75%           max  
server_id                                                          
1          1.508436e+09  1.508436e+09  1.508436e+09  1.508437e+09  
2          1.508436e+09  1.508436e+09  1.508436e+09  1.508437e+09  
3          1.508436e+09  1.508436e+09  1.508436e+09  1.508437e+09  
4          1.508436e+09  1.508436e+09  1.508436e+09  1.508437e+09  
5          1.508436e+09  1.508436e+09  1.508436e+09  1.508437e+09  
6          1.508436e+09  1.508436e+09  1.508436e+09  1.508437e+09  
7          1.508436e+09  1.508436e+09  1.508436e+09  1.508437e+09  
8          1.508436e+09  1.508436e+09  1.508436e+09  1.508437e+09  
9          1.508436e+09  1.508436e+09  1.508436e+09  1.508437e+09  
10         1.508436e+09  1.508436e+09  1.508436e+09  1.508437e+09

In [11]:
p=pd.Series()
for i in range(1,11):    
    s=df.groupby(['server_id'])['start_timestamp'].get_group(i).diff().shift(-1)
    p=p.append(s)
p.sort_index()
df['app_time']=p

In [12]:
df['app_time'].describe()

count    990.000000
mean       9.951515
std        9.424152
min        0.000000
25%        3.000000
50%        7.000000
75%       14.000000
max       61.000000
Name: app_time, dtype: float64

In [13]:
df['app_time'].isnull().sum()

10

In [14]:
df_filtered=df[(df.app_time > 45 ) & (df.app_time < 300)]
df_filtered

,server_id,application_id,start_timestamp,app_time
360,10,app9,1508435914,59.0
528,10,app4,1508436073,56.0
608,7,app2,1508436169,61.0
657,7,app4,1508436232,52.0
823,1,app7,1508436396,48.0
832,9,app8,1508436401,47.0


In [15]:
df['application_id'].groupby(df['server_id']).describe()

,count,unique,top,freq
server_id,,,,
1,89,10,app10,14
2,122,10,app7,18
3,98,10,app4,18
4,107,10,app8,13
5,93,10,app9,18
6,108,10,app8,14
7,86,10,app1,13
8,100,10,app1,15
9,105,10,app8,14


In [16]:
df2=df.groupby(['server_id', 'application_id'])['app_time'].describe(percentiles=[.1])
df2.head()

count       mean       std  min  10%   50%   max
server_id application_id                                                  
1         app1              7.0   8.857143  7.515064  0.0  0.6  13.0  17.0
          app10            14.0   9.857143  8.627836  0.0  1.0   8.0  30.0
          app2              8.0   9.500000  7.191265  2.0  2.7   8.0  23.0
          app3             11.0  11.181818  8.518429  1.0  3.0   8.0  27.0
          app4              4.0   9.500000  6.557439  4.0  4.9   7.5  19.0

In [17]:
df2.columns

Index(['count', 'mean', 'std', 'min', '10%', '50%', 'max'], dtype='object')

In [18]:
REF=pd.DataFrame(df2['10%'])

In [19]:
REF.xs(1, level='server_id')

,10%
application_id,
app1,0.6
app10,1.0
app2,2.7
app3,3.0
app4,4.9
app5,2.4
app6,1.0
app7,4.5
app8,1.0


## Visualization

In [20]:
#from bokeh.charts import Bar, output_file, show
from bokeh.io import show, output_file
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure
from bokeh.palettes import Spectral10
from bokeh.models import HoverTool

In [21]:
df3=df.groupby(df['server_id'])[['application_id', 'start_timestamp', 'app_time']]

In [22]:
Spectral10

['#5e4fa2',
 '#3288bd',
 '#66c2a5',
 '#abdda4',
 '#e6f598',
 '#fee08b',
 '#fdae61',
 '#f46d43',
 '#d53e4f',
 '#9e0142']

In [23]:
output_file("bar_colors.html")
# Qua uso come esempio il Server 1. 
# Sostituendo nelle due righe successive all'1 un indice si può fare un ciclo sui server.
# Oppure si può costruire una funzione che ha come input il numero del server.
source = ColumnDataSource(df3.get_group(1))
REF_app=REF.xs(1, level='server_id')



colormap = {'app1':'#5e4fa2', 'app2': '#3288bd','app3': '#66c2a5', 'app4': '#abdda4', \
            'app5': '#e6f598', 'app6': '#fee08b', 'app7': '#fdae61', 
            'app8': '#f46d43', 'app9': '#d53e4f', 'app10': '#9e0142'}
colors = [colormap[x] for x in source.data['application_id']]

hover = HoverTool(tooltips=[
    ("Application", "@application_id"),
    ("(x,y)", "($x, $y)"),
],             
    mode='hline'
                 )


x_label = 'Start Time (timestamps)'
y_label = 'Total time (seconds)'
title = 'Server %d' % i

xrange=(source.data['start_timestamp'].min(),source.data['start_timestamp'].max())
yrange=(source.data['app_time'].min(),source.data['app_time'].max())

pp = figure(x_range=xrange, y_range=yrange,
            x_axis_label = x_label, y_axis_label = y_label, title=title,\
           plot_width=1000, plot_height=1000)

pp.vbar(source=source, x='start_timestamp', top='app_time', width=1, \
        color=colors)
for j in range(1,11):
    pp.line(x=source.data['start_timestamp'], y=REF_app.get_value('app'+str(j),'10%'),\
    color=colormap['app'+str(j)], line_width=2, alpha=1)

    
pp.title.text_color = "olive"
pp.title.text_font = "helvetica"
pp.title.text_font_style = "bold"    
    
labels = LabelSet(x='start_timestamp', y=REF_app, text='application_id', y_offset=8,
                  text_font_size="10pt", text_color="#555555",
                  source=source, text_align='center')
pp.add_layout(labels)    
    
show(pp)

/anaconda/lib/python3.6/site-packages/bokeh/util/deprecation.py:34: BokehDeprecationWarning: 
Supplying a user-defined data source AND iterable values to glyph methods is deprecated.

See https://github.com/bokeh/bokeh/issues/2056 for more information.

  warn(message)
/anaconda/lib/python3.6/site-packages/bokeh/util/deprecation.py:34: BokehDeprecationWarning: 
Supplying a user-defined data source AND iterable values to glyph methods is deprecated.

See https://github.com/bokeh/bokeh/issues/2056 for more information.

  warn(message)


ValueError: expected an element of either String, Dict(String, Either(String, Instance(Transform), Instance(ColorMapper), Float)) or Float, got                 10%
application_id     
app1            0.6
app10           1.0
app2            2.7
app3            3.0
app4            4.9
app5            2.4
app6            1.0
app7            4.5
app8            1.0
app9            3.4